In [ ]:
%matplotlib notebook
import Augmentor
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms, datasets
from tqdm import tqdm_notebook
from PIL import Image


'''Data Processing'''
#importing train data and labels
data_training = scipy.io.loadmat('Augmented_images.mat')
TrainingPatches = data_training['TrainingPatches']
TrainingGTLabels = data_training['TrainingGTLabels'][0]

#import test data and labels
data_testing = scipy.io.loadmat('FullTestData.mat')
TestPatches = data_testing['TestPatches']
TestGTLabels = data_testing['TestGTLabels'][0]

# convert train data shape from [x,y, len] to  [len, 1, x, y]
inputs, train_labels = TrainingPatches, TrainingGTLabels
inputs = np.transpose(inputs, (2, 0, 1))
inputs = np.expand_dims(inputs, 1)
inputs = torch.tensor(inputs, dtype=torch.float32)

# convert test data shape from [x,y, len] to  [len, 1, x, y]
outputs, test_labels = TestPatches, TestGTLabels
outputs = np.transpose(outputs, (2, 0, 1))
outputs = np.expand_dims(outputs, 1)
outputs = torch.tensor(outputs, dtype=torch.float32)

inputs = inputs.view(inputs.size(0), 1, inputs.size(1), inputs.size(2), inputs.size(3))
outputs = outputs.view(outputs.size(0), 1, outputs.size(1), outputs.size(2), outputs.size(3))

'''Data Loader V1: small trial set of images'''

class GLAUCOMA_small(torch.utils.data.Dataset): #include data in class
   
    def __init__(self, train_path, transform=None):

        #importing train data and labels
        data_training = scipy.io.loadmat(train_path)
        TrainingPatches = data_training['TrainingPatches']
        TrainingGTLabels = data_training['TrainingGTLabels'][0]
        

        #converting labels from 1,-1 to 1,0
        test_labels = np.zeros((len(TrainingGTLabels), 2))

        test_labels[:,0] = (TrainingGTLabels==-1)*1
        test_labels[:,1] = (TrainingGTLabels==1)*1

        data_training['TrainingGTLabels'] = test_labels
        
        TrainingGTLabels = data_training['TrainingGTLabels'][:,1]
        
        # convert train data shape from [30,30, 62022] to  [62022, 1, 30, 30]
        inputs, train_labels = TrainingPatches, TrainingGTLabels
        inputs = np.transpose(inputs, (2, 0, 1))
        inputs = np.expand_dims(inputs, 1)
        
        # 900 training images / labels for equal no of class samples (4500 and 4500, 9000 just looks nice)
        small_train_data = inputs#[0:9000]
        small_train_labels = train_labels#[0:9000]
        
        self.transform = transform
        
        self.train_data = small_train_data
        self.train_labels = small_train_labels
        print('shape of data:', np.shape(self.train_data))


    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        # return right arguments if test, train supervised, train unsupervised
        img, target = self.train_data[index], self.train_labels[index]
        
        #optional transform
#         if self.transform != None:
#         for trf in self.transform:
#             if trf == 'rotations':
#                 ones = small_train_data[small_train_labels == 1,:,:,:]
#                 first = np.zeros(small_train_data.shape)
#                 for i in range(small_train_data.shape[0]):
#                     first[i,:,:,:] = np.rot90(ones)

#         img = Image.fromarray(img.astype('uint8'))
#         if self.transform is not None:
#             img = self.transform(img)
#             img = np.array(img)

        return img, target

    def __len__(self):
        return len(self.train_data)
        
temp = GLAUCOMA_small('Augmented_images.mat')

trainloader = torch.utils.data.DataLoader(temp, batch_size=128, shuffle=True)

temp = GLAUCOMA_small('FullTestData_NEW.mat')

testloader = torch.utils.data.DataLoader(temp, batch_size=128, shuffle=True)

'''Define Initial Weights'''

train_labels = scipy.io.loadmat('Augmented_images.mat')['TrainingGTLabels'].squeeze()
w = float(sum(train_labels == 1))/sum(train_labels==-1)

'''Define CNN Architecture'''


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 20, 3, 1)
        # Max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 16, 2, 1)
        self.conv3 = nn.Conv2d(16, 16, 2, 1)
        #self.conv4 = nn.Conv2d(16, 16, 2, 1)
        
        # affine operation: y = Wx + b
        self.fc1 = nn.Linear(576, 64)
        self.fc2 = nn.Linear(64, 500)
        self.fc3 = nn.Linear(500, 120)
        self.fc4 = nn.Linear(120, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        print(x.shape)
        x = self.pool(F.relu(self.conv3(x)))
        print(x.shape)
        #x = self.pool(F.relu(self.conv4(x)))
        print(x.shape)
        
        x = x.view(x.size(0),-1)
        #x = x.view(-1, 16 * 6 * 6)
        print(x.shape)
        x = F.relu(self.fc1(x))
        print(x.shape)
        x = F.relu(self.fc2(x))
        print(x.shape)
        x = F.relu(self.fc3(x))
        print(x.shape)
        #x = F.softmax(self.fc4(x))
        x = F.softmax(F.relu(self.fc4(x)))
        print(x.shape)
        return x
        
        
net = Net()

                                                                                                                        
'''define loss function and optimiser'''
import torch.optim as optim

criterion = nn.CrossEntropyLoss(weight=torch.Tensor([w,1]))
optimizer = optim.Adam(net.parameters(), lr=0.001)      

#train the network, feed the inputs to the network, optimise

for epoch in range(2): #loop over the dataset
     
    running_loss = 0.0
    all_outputs = []

    for i, data in tqdm_notebook(enumerate(trainloader, 0)):
        inputs, labels = data
        
        inputs = inputs.type('torch.FloatTensor')
        labels = labels.type('torch.LongTensor')
        
        optimizer.zero_grad()
        
        #forward + backward + optimise
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        all_outputs.append(outputs)
        
        #print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

'''Test Network'''

#datacheck
dataiter = iter(testloader)
images, labels = dataiter.next()
print(images.size())
print(labels.size())

correct = 0
total = 0
with torch.no_grad():
    y_pred = []
    all_scores = []
    first = True
    for data in tqdm_notebook(testloader):
        images, labels = data
        images = images.type('torch.FloatTensor')
        labels = labels.type('torch.LongTensor')())
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        scores = outputs.data[:,1].numpy()
        y_pred.append(predicted.numpy())
        all_scores.append(scores)
        
        if first:
            first = False
            y_actual = labels.numpy()
        else:
            y_actual = np.concatenate((y_actual, labels.numpy()))
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on test images: %d %%' % (
    100 * correct / total))
    
'''Confusion Matrix and F score'''

new_scores = all_scores[0]

for i in range(1, len(all_scores)):
    new_scores = np.concatenate((new_scores, all_scores[i]))
    
new_y_pred = y_pred[0]

for i in range(1, len(y_pred)):
    new_y_pred = np.concatenate((new_y_pred, y_pred[i]))
    

#ouputting confmat, precsion and recall
from sklearn.metrics import confusion_matrix, recall_score, precision_score

print(confusion_matrix(y_actual, new_y_pred))

(recall_score(y_actual, new_y_pred), precision_score(y_actual, new_y_pred))

#use scikit to find average precision-recall score and F1 score
from sklearn.metrics import average_precision_score

average_precision = average_precision_score(y_actual, new_scores)

print('Average precision-recall score: %f' % average_precision)

F1=2*((recall_score(y_actual, new_y_pred))*(precision_score(y_actual, new_y_pred)))/((recall_score(y_actual, new_y_pred))+(precision_score(y_actual, new_y_pred)))
print('F1 score: %f' % F1)

'''ROC curve'''
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
%matplotlib inline

precision, recall, _ = precision_recall_curve(y_actual, new_scores)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))
plt.savefig('C:/Users/mark/Desktop/PR_curve_augmented1500_2.png')
plt.show()

#to see how the two individual classes performed
classes = ('pores', 'non-pores')
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.type('torch.FloatTensor')
        labels = labels.type('torch.LongTensor')
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print(class_total)
print(class_correct)
#gives output for each class
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
classes[i], 100 * class_correct[i] / class_total[i]))